In [22]:
from pinance import Pinance

In [23]:
symbol = "AAPL"

stock = Pinance(symbol)

In [27]:
news = stock.get_news() 

# Bert Model

In [31]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import transformers as ppb # pytorch transformers

I0519 11:54:20.648908 4583550400 file_utils.py:41] PyTorch version 1.2.0 available.
I0519 11:54:25.644062 4583550400 file_utils.py:57] TensorFlow version 2.1.0 available.


In [32]:
# Using DistillBert here to get a quick baseline
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# model = model_class.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained('bert-base-uncased')

I0519 11:54:29.965715 4583550400 modeling.py:580] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/anurag/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
I0519 11:54:29.967367 4583550400 modeling.py:588] extracting archive file /Users/anurag/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/_p/9dls8ssn77z2q_b4r1c5pbhh0000gn/T/tmp6seb_t0t
I0519 11:54:35.013293 4583550400 modeling.py:598] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
 

In [33]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def dataProcessingEmbedding(Dataset):
  tokenized_text = []
  indexed_tokens = []
  segments_ids = []
  marked_text_lis = []
  error_lis = []
  for i in range(0,len(Dataset)):

    marked_text = "[CLS] "+str(Dataset)+" [SEP]" # a window of previous 20 words. For the first 20 words isn't as much words as possible. hence the very last first one is just a single word. 

    try:
      # Tokenize our sentence with the BERT tokenizer.
        tokens = tokenizer.tokenize(marked_text)
        marked_text_lis.append(marked_text)
        tokenized_text.append(tokens)
        indexed_tokens.append(tokenizer.convert_tokens_to_ids(tokens))
        segments_ids.append([1] * len(tokens))
    except:
        error_lis.append(i)

  return tokenized_text, indexed_tokens, segments_ids, marked_text_lis

I0519 11:55:01.059229 4583550400 tokenization.py:190] loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/anurag/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [ ]:
tokenized_text, indexed_tokens, segments_ids, marked_text_lis = dataProcessingEmbedding(news)

In [ ]:
# Convert inputs to PyTorch tensors
def convertTokens_Tensors(indexed_tokens,segments_ids):
  pad = len(max(indexed_tokens, key=len))
  indexed_tokens_pad = np.array([i + [0]*(pad-len(i)) for i in indexed_tokens])
  segments_ids_pad = np.array([i + [1]*(pad-len(i)) for i in segments_ids])

  tokens_tensor = torch.Tensor(indexed_tokens_pad)
  segments_tensors = torch.tensor(segments_ids_pad)

  return tokens_tensor, segments_tensors

In [ ]:
tokens_tensor, segments_tensors = convertTokens_Tensors(indexed_tokens,segments_ids)

In [ ]:
with torch.no_grad():
  encoded_layers,_ = model(tokens_tensor.to(torch.int64), attention_mask=segments_tensors.to(torch.int64))
  token_embeddings = torch.stack(encoded_layers, dim=0)

  semanticseqs = torch.mean(torch.sum(token_embeddings[-4:,:,:,:], dim=0),dim=1).numpy()

# Classification Model